In [ ]:
import pyreadstat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_plink
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperModel, RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy import stats
import warnings

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable GPU

import tensorflow as tf
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2025-10-03 13:42:56.223604: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-03 13:42:56.225931: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-03 13:42:56.229090: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-03 13:42:56.235041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759513376.247607 3578329 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759513376.25

In [2]:
# Initialization parameters

# Training CEDS scale
# Choose the target data A = 19 predictors or B = 10 predictors
#CESD_type = 'B' # 'A' or 'B'

# Choose the target data Wave: Wave I or Wave IV
#test_wave = 'Wave I' # 'Wave I' or 'Wave IV'

# Model single (only one neural network is trained) 
# Model multiple (several differents neural networks are trained and the best one is chosen)
# DO NOT FORGET IF YOU ARE USING MULTIPLE TO CHECK IF THERE IS A FOLDER CALLED hyperparameters-log
# IF THERE IS THE MODEL WILL NOT TRAIN PROPERLY
#model_type = 'single'

# Include Genetic data? (boolean)
#include_genetic_data = True

# Account for ancestry
#control_ancestry = True # Error when controling for ancestry I dont know why

# Account for race
#control_race = False

# Choose sex
#sex_gender = None # 'Male' or 'Female' or None

info_dict = {
    "CESD_type": "A",  # 'A' (19 predictors) or 'B' (10 predictors)
    "test_wave": 'Wave I',  # 'Wave I' or 'Wave IV' for the first wave
    "model_type": "single",  # 'single' (one NN) or 'multiple' (best of several NNs)
    "include_genetic_data": False,  # Include genetic data or not
    "control_ancestry": True,  # Account for ancestry (error noted)
    "control_race": False,  # Account for race
    "sex_gender": None,  # 'Male', 'Female', or None
    'include_environmental_data': True, # Include environmental data or not
    'include_depressive_symptoms': False, # Include depressive symptos
    'clinically_depressed': None # adolescence or adulthood or None (also need to put test_wave = Wave IV)
}

In [3]:
# Functions

import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

def bootstrap_metrics(y_true, y_prob, n_bootstraps=1000, ci=95):
    """Computes percentile bootstrap confidence intervals for AUC."""
    aucs = []
    accuracies = []

    # For a sigmoid output, threshold at 0.5 to get binary predictions
    y_pred = (y_prob >= 0.5).astype(int)

    # Generate bootstrap samples
    for _ in range(n_bootstraps):
        # Resample indices with replacement
        indices = resample(range(len(y_true)), replace=True)
        y_true_boot = np.array(y_true)[indices]
        y_prob_boot = np.array(y_prob)[indices]
        y_pred_boot = np.array(y_pred)[indices]
        
        # Compute AUC for this bootstrap sample
        aucs.append(roc_auc_score(y_true_boot, y_prob_boot))
    
        # Compute Accuracy for this bootstrap sample
        accuracies.append(accuracy_score(y_true_boot, y_pred_boot))

    # Sort the AUC values
    aucs = np.sort(aucs)
    # Sort the accuracy values
    accuracies = np.sort(accuracies)
    
    # Compute percentiles for CI
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    
    lower_bound_auc = np.percentile(aucs, lower_percentile)
    upper_bound_auc = np.percentile(aucs, upper_percentile)

    lower_bound_acc = np.percentile(accuracies, lower_percentile)
    upper_bound_acc = np.percentile(accuracies, upper_percentile)
    
    return aucs, np.mean(aucs), lower_bound_auc, upper_bound_auc, np.mean(accuracies), lower_bound_acc, upper_bound_acc

def plot_confusion_matrix_and_roc(y_true, model, X_test, title,info_dict, threshold=0.5):
    """
    Plots confusion matrix on the left and ROC curve with AUC on the right.

    Parameters:
    - y_true: Actual labels.
    - model: Trained model to generate predicted probabilities for ROC curve.
    - X_test: Test features, required to calculate predicted probabilities.
    - title: Title for the plot.
    - threshold: The threshold to apply on predicted probabilities for classification (default is 0.5).
    - info_dict: Information about the data, use to plot different colors
    """
    # 1. Predict probabilities for the test set (needed for ROC curve and prediction)
    y_prob = model.predict(X_test)
    
    # 2. For a sigmoid output, threshold at 0.5 to get binary predictions
    y_pred = (y_prob >= threshold).astype(int)
    
    # 3. Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # 4. Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # 5. Compute ROC and AUC
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)

    # 6. Create subplots: left for confusion matrix, right for ROC curve
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))  # Create 1 row, 2 columns layout

    # Colors
    
    if info_dict['sex_gender'] == 'Female':
        cmaps = 'Reds'
        colors = 'r' 
    elif info_dict['sex_gender'] == 'Male':
        cmaps = 'Blues'
        colors = 'b'
    else: 
        cmaps = 'gray'
        colors = 'k'
        
    # Compute AUC with bootstrap confidence intervals
    aucs, auc_mean, auc_lower, auc_upper, acc_mean, acc_lower, acc_upper = bootstrap_metrics(y_true, y_prob)
    print(f"AUC: {auc_mean:.3f} (CI: {auc_lower:.3f} - {auc_upper:.3f})")
    print(f"accuracy: {acc_mean:.3f} (CI: {acc_lower:.3f} - {acc_upper:.3f})")    
        
    # Left plot: Confusion Matrix
    ax1 = axes[0] 
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmaps, annot_kws={'size': 16}, 
                xticklabels=['Does not have\nsymptoms of depression (0)', 
                             'Has symptoms of\n depression (1)'], 
                yticklabels=['Does not have\nsymptoms of depression (0)', 
                             'Has symptoms of\n depression (1)'], 
                cbar_kws={'label': 'Count'}, linewidths=1, linecolor='gray', 
                vmin=0, vmax=np.max(cm), ax=ax1)
    ax1.set_xlabel('Predicted', fontsize=12)
    ax1.set_ylabel('Actual', fontsize=12)
    
    # Add metrics text next to the confusion matrix
    metrics_text = f"Accuracy: {accuracy:.2f}\nRecall: {recall:.2f}\nF1 Score: {f1:.2f}"
    ax1.text(.8, 1, metrics_text, fontsize=12, ha='left', va='center', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))

    # Right plot: ROC Curve
    ax2 = axes[1]
    ax2.plot(fpr, tpr, color=colors, label=f'ROC curve (AUC = {roc_auc:.2f})')
    ax2.plot([0, 1], [0, 1], color='gray', linestyle='--')
    ax2.set_xlabel('False Positive Rate', fontsize=12)
    ax2.set_ylabel('True Positive Rate', fontsize=12)
    ax2.set_title('Receiver Operating Characteristic (ROC) Curve', fontsize=14)
    ax2.legend(loc='lower right')

    # Set the plot title
    fig.suptitle(title, fontsize=16)

    # Show the plot
    plt.tight_layout()
    plt.show()

    return aucs, auc_mean, auc_lower, auc_upper, acc_mean, acc_lower, acc_upper

In [4]:
## To list all folders and files in directory

def find_files_folders(directory_path):
    files_and_folders = os.listdir(directory_path)
    print(files_and_folders)
    

# Some of the test group in WAVE I are not present in wave IV (I can change to make only the training in wave I and test in WAVE IV to compare)
)

In [5]:
# check if CUDA is avaliable

print(tf.__version__)  # Print the TensorFlow version

# Check CUDA and cuDNN versions used by TensorFlow
# Check if TensorFlow can access CUDA-enabled GPU
if tf.config.list_physical_devices('GPU'):
    print("CUDA is available! TensorFlow is using the GPU.")
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    from tensorflow.python.platform import build_info as tf_build_info
    print("CUDA Version: ", tf_build_info.cuda_version)
    print("cuDNN Version: ", tf_build_info.cudnn_version)
else:
    print("CUDA is not available. TensorFlow is not using the GPU.")


2.19.0
CUDA is not available. TensorFlow is not using the GPU.


2025-10-03 13:42:58.131687: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Wave 1

## Environmental

In [ ]:
# Load the .xpt file
df1, meta1 = pyreadstat.read_xport('REPLACE_WITH_YOUR_WAVE_I_FILE_LOCATION') # file location
df1.shape

(20745, 83)

In [ ]:
# Get the data types
dtypes = df1.dtypes

# Convert to DataFrame
dtypes_df = dtypes.reset_index()
dtypes_df.columns = ['Column Name', 'Data Type']

# Save to CSV
dtypes_df.to_csv('data_types.csv', index=False)

# count
dtypes.value_counts()

In [ ]:
# Rename columns for better explanation

df1.rename(columns={'BIO_SEX_numeric': 'SEX', # 1 males 2 females
                  'race_recoded_numeric':'RACE',
                  'PA55':'INCOME',
                  'PA56_numeric':'ENOUGH MONEY FOR BILLS',
                  'PA59_numeric':'MEDICAL CARE FOR FAMILY',
                  'H1NB5_numeric':'FEEL SAFE IN NBORHOOD?',
                  'H1NB6_numeric':'HOW HAPPY LIVING IN NBORHOOD',
                  'H1GH1_numeric':'GENERAL HEALTH',
                  'H1PL1_numeric':'DIFFICULTY USING HANDS/FT/LIMBS',
                  'H1GH2_numeric':'FREQ-HEADACHES',
                  'H1GH3_numeric':'FREQ-FEELING HOT',
                  'H1GH4_numeric':'FREQ-STOMACHACHE',
                  'H1GH5_numeric':'FREQ-COLD SWEATS',
                   'H1GH6_numeric':'FREQ-FEELING PHYSICALLY WEAK', # 0 never / 1 few times / 2 once a week / 3 almost every day / 4 every day
                   'H1GH7_numeric':'FREQ-SORE THROAT/COUGH',
                   'H1GH8_numeric':'FREQ-VERY TIRED FOR NO REASON',
                   'H1GH9_numeric':'FREQ-PAINFUL/OFTEN URINATION',
                   'H1GH10_numeric':'FREQ-FEELING VERY SICK',
                   'H1GH11_numeric':'FREQ-WAKE UP FEELING TIRED',
                   'H1GH12_numeric':'FREQ-SKIN PROBLEMS',
                   'H1GH13_numeric':'DIZZINESS',
                   'H1GH14_numeric':'CHEST PAINS',
                   'H1GH15_numeric':'FREQ-MUSCLE/JOINT ACHES/PAINS',
                   'H1GH16_numeric_recode':'FREQ-MENSTRUAL CRAMPS',
                   'H1GH17_numeric':'FREQ-POOR APPETITE',
                   'H1GH18_numeric':'FREQ-INSOMNIA',
                   'H1GH19_numeric':'FREQ-TROUBLE RELAXING',
                   'H1GH22_numeric':'FREQ-FEARFULNESS',
                   'H1GH26_numeric':'NEEDED BUT NOT GET MEDICAL CARE',
                   'H1GH27I_numeric_recode':'WHY-COULD NOT PAY',
                   'H1GH28_numeric':'WEIGHT IMAGE',
                   'H1GH29_numeric':'TO GAIN/LOSE/MAINTAIN WEIGHT',
                   'H1HS3_numeric':'PSYCHOLOGICAL COUNSELING',
                   'H1GH48_numeric':'HEALTH CAUSE SCHOOL ABSENCE',
                   'H1GH49_numeric':'HEALTH CAUSE SOCIAL ABSENCE',
                   'H1ED15_numeric':'TROUBLE-GETTING ALONG TEACHERS',
                   'H1ED16_numeric':'TROUBLE-PAYING ATTENTION SCHOOL',
                   'H1ED17_numeric':'TROUBLE-GETTING HOMEWORK DONE',
                   'H1ED18_numeric':'TROUBLE-WITH OTHER STUDENTS',
                   'H1ED19_numeric':'FEEL CLOSE TO PEOPLE AT SCHOOL',
                   'H1ED20_numeric':'FEEL PART OF YOUR SCHOOL',
                   'H1ED21_numeric':'STUDENTS AT SCHOOL PREJUDICED',   # 1 strong agree / 2 agree / 3 neutral / 4 disagree / 5 strong disagree
                   'H1ED22_numeric':'HAPPY AT YOUR SCHOOL', # 1 strong agree / 2 agree / 3 neutral / 4 disagree / 5 strong disagree
                   'H1ED23_numeric':'TEACHERS TREAT STUDENTS FAIRLY',
                   'H1ED24_numeric':'FEEL SAFE IN YOUR SCHOOL',
                   'H1DA7_numeric':'HANG OUT WITH FRIENDS',
                   'H1WP9_numeric':'CLOSE TO MOM',
                   'H1WP10_numeric':'MOM-HOW MUCH DOES SHE CARE',
                   'H1WP13_numeric':'CLOSE TO DAD',
                   'H1WP14_numeric':'DAD-HOW MUCH DOES HE CARE',
                   'H1PF1_numeric':'MOM-WARM AND LOVING',
                   'H1PF4_numeric':'MOM-GOOD COMMUNICATION',
                   'H1PF5_numeric':'MOM-GOOD RELATIONSHIP',
                   'H1PF24_numeric':'DAD-GOOD COMMUNICATION',
                   'H1PF25_numeric':'DAD-GOOD RELATIONSHIP',
                   'H1PR1_numeric':'ADULTS CARE ABOUT',
                   'H1PR2_numeric':'TEACHERS CARE ABOUT YOU',
                   'H1PR3_numeric':'PARENTS CARE ABOUT YOU',
                   'H1PR4_numeric':'FRIENDS CARE ABOUT YOU',
                   'H1PR5_numeric':'FAMILY UNDERSTAND YOU',
                   'H1PR6_numeric':'WANT TO LEAVE HOME',
                   'H1PR7_numeric':'FAMIYL HAS FUN TOGETHER',
                   'H1PR8_numeric':'FAMILY PAYS ATTENTION TO YOU',
                   'H1PF33_numeric':'LIKE SELF AS ARE',
                   'H1PF34_numeric':'DO EVERYTHING JUST RIGHT',
                   'H1PF35_numeric':'FEEL SOCIALLY ACCEPTED',
                   'H1PF36_numeric':'FEEL LOVED AND WANTED',
                   'H1CO10F_numeric_recode':'FORCED SEX',
                   'H1NR3_numeric':'SEX FOR DRUGS OR MONEY',
                   'H1FV1_numeric':'SAW SHOOTING/STABBING OF PERSON',
                   'H1FV2_numeric':'HAD KNIFE/GUN PULLED ON YOU',
                   'H1FV3_numeric':'SOMEONE SHOT YOU',
                   'H1FV4_numeric':'SOMEONE STABBED YOU',
                   'H1FV5_numeric':'GOT INTO A PHYSICAL FIGHT',
                   'H1FV6_numeric':'WERE JUMPED',
                   'H1FP21_numeric_recode':'PREG1 PREGNANCY OUTCOME',
                   'H1SU4_numeric':'PAST YR-FRIENDS ATTEMPT SUICIDE',
                   'H1SU5_numeric_recode':'PAST YR-FRIENDS SUCCEED',
                   'H1SU6_numeric':'PAST YR-FAMILY ATTEMPT SUICIDE',
                   'H1SU7_numeric_recode':'PAST YR-FAMILY SUCCEED'
                  }, inplace=True)

In [ ]:
print(df1.shape)
df1.head()

In [ ]:
# Remove depressive symptoms if needed

if not info_dict.get("include_depressive_symptoms", True):
    columns_to_remove_symp = [
        "FREQ-POOR APPETITE",             # Question 5
        "FREQ-TROUBLE RELAXING",          # Question 7
        "FREQ-FEARFULNESS",
        "FREQ-VERY TIRED FOR NO REASON"   # Question 4
    ]
    df1 = df1.drop(columns=[col for col in columns_to_remove_symp if col in df1.columns])
print(df1.shape)

## Genetic

In [ ]:
## Skipped for now

# Genetic information path
#path = '/pdfs/rche/addhealth/AH_gwas/'
#dir_list = os.listdir(path)
#print("Files and directories in '", path, "' :")
# prints all files
#print(dir_list)

In [ ]:
gene4, meta_gene4 = pyreadstat.read_xport("REPLACE_WITH_PGS_LOCATION")

print('Ancestry')
print(gene4['PSANCEST'].value_counts())

print('Race')
print(gene4['PSRACE'].value_counts())


In [ ]:
# Access metadata
print("Variable Names and Labels:")
for var_name, label in zip(meta_gene4.column_names, meta_gene4.column_labels):
    print(f"{var_name}: {label if label else 'No label'}")

In [ ]:
# Extracting value counts
ancest_counts = gene4['PSANCEST'].value_counts()
race_counts = gene4['PSRACE'].value_counts()

# Creating dictionaries to store value counts for conversion
ancest_dict = ancest_counts.to_dict()
race_dict = race_counts.to_dict()

# Remove the last entry from race_dict by slicing
race_dict = dict(list(race_dict.items())[:-1])

# Remove the last entry from race_counts
race_counts = race_counts.iloc[:-1]

# Printing the dictionaries
print("Ancestor Counts:", ancest_dict)
print("Race Counts:", race_dict)

# Plotting the pie chart for PSANCEST with adjusted label positions
plt.figure(figsize=(8, 6))
wedges, texts, autotexts = plt.pie(ancest_counts, labels=ancest_counts.index, autopct='%1.1f%%', startangle=90,
                                   wedgeprops=dict(edgecolor='black', linewidth=1.5), textprops=dict(color='black', fontsize=10),
                                   labeldistance=1.1)

plt.title('Ancestry',fontsize=16)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()  # Adjust layout to avoid overlap
plt.show()

# Plotting the pie chart for PSRACE with a different labeldistance
plt.figure(figsize=(8, 6))
wedges, texts, autotexts = plt.pie(race_counts, labels=race_counts.index, autopct='%1.1f%%', startangle=90,
                                   wedgeprops=dict(edgecolor='black', linewidth=1.5), textprops=dict(color='black', fontsize=12),
                                   labeldistance=1.1)

# Manually adjusting the label positions for overlap
for text in texts:
    text.set_fontsize(10)
    text.set_color('black')

# Adjusting label positions
texts[3].set_position((.17, 1.05))  # Move "Asian" further out
texts[4].set_position((-0.15, 1.06))  # Move "Nat. Am." further out

# Adjusting percentage positions
autotexts[3].set_position((.11, 0.6))  # Move percentage for "Asian" further out
autotexts[4].set_position((-.10, 0.6))  # Move percentage for "Nat. Am." further out

plt.title('Race',fontsize=16)
plt.axis('equal')
plt.tight_layout()  # Adjust layout to avoid overlap
plt.show()


In [ ]:
# Droping unused collumns
imputed_gene4 = gene4
# Label Encoding
label_encoder = LabelEncoder()

# Account for ancestry
if info_dict['control_ancestry']:
    imputed_gene4 = imputed_gene4[imputed_gene4['PSANCEST'] == 'European ancestry']
    imputed_gene4 = imputed_gene4.drop(columns=['PSANCEST'])
    imputed_gene4 = imputed_gene4.reset_index(drop=True)
else:
    # Encoding the categorical column ancestry
    imputed_gene4['PSANCEST'] = label_encoder.fit_transform(imputed_gene4['PSANCEST'])
    
# Account for race
if info_dict['control_race']:
    imputed_gene4 = imputed_gene4[imputed_gene4['PSRACE'] == 'NHW']
    imputed_gene4 = imputed_gene4.drop(columns=['PSRACE'])
    imputed_gene4 = imputed_gene4.reset_index(drop=True)
else:
    # Encoding the categorical column race
    imputed_gene4['PSRACE'] = label_encoder.fit_transform(imputed_gene4['PSRACE'])

imputed_gene4.columns.tolist()

In [ ]:
# website: https://addhealth-navigator.cpc.unc.edu/data/example.org/a6259610-271b-40f2-aca5-d4d22251a875/example.org/e7c8a7d8-df14-4e49-84c8-d2ac77256ac5/example.org/d79b1b15-2942-469e-87d1-fe045aa91757
dictw = {
    "FID": "FID",
    "AID": "AID",
    "PSCADSTD": "CARDIoGRAM Coronary Artery Disease",
    "PSBMI1":"GIANT BMI, 2018",
    "PSADSTD": "Alzheimer's Diease, 2019",
    "PSEDU1":"Educational Attainment, 2018",
    "PSAUTISM":"Autism Spectrum Disorder, 2017",
    "PSEVRSK1":"Ever Regular Smoker, 2019",
    "PSADEA":"CTG Alcohol Dependence, 2019",
    "PSMENARC":"Menarche, 2014",
    "PSMENO":"Menopause, 2015",
    "PSPTSDAL":"PTSD, 2018", # sub: all (x) - EA - AA
    "PSADHD2":"Attention-deficit/hyperactivity disorder, 2017",
    "PSBIPOL":"Bipolar, 2011",
    "PSSCZSTD":"Schizophrenia, 2011",
    "PSMDD1":"Major Depressive Disorder, 2019",
    "PSPOSAF1":"Wellbeing - Positive Affection, 2019", # sub: Dep - LifeSat - Neuroticism - PosAff (x)
    "PSEXTRAV":"Extraversion, 2015",
    "PSNEURO1":"Neuroticism, 2019",
    "PSANSTD":"Anorexia Nervosa, 2019", # 90% sure
    "PSSOCSTD":"Social Isolation, 2018", # PGC_Social_2018_PubClub PGC_Social_2018(x)
    "PSLONELY":"Lonely, 2018",
    "PSINSOMN":"Insomnia, 2019", # sub: nosub(x) - Dozing - Morning - Napping - Snoring - GetUp - SleepDur
}

# Step 1: Filter columns
imputed_gene4 = imputed_gene4[[col for col in imputed_gene4.columns if col in dictw]]

# Step 2: Rename columns
imputed_gene4.rename(columns=dictw, inplace=True)

# Output the resulting DataFrame
imputed_gene4.head()

# Getting the depression score

In [ ]:
# CESD_W1A is using 19 questions
# CESD_W1B is using 10 questions
df1.shape

A score of 10 or greater indicates symptoms of depression; 

16 or higher indicates severe symptoms of depression.

Obs: we are using the incomplete version of the test, so we need to correct for that

In [ ]:
from scipy.stats import kurtosis
def plot_histogram_with_stats(data, column_name, line_at=16):
    """
    Plots a histogram of the specified column and calculates basic statistics, 
    including mean, standard deviation, skewness, and kurtosis. A vertical dashed 
    line is drawn at the specified location.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    column_name (str): The name of the column to be analyzed and plotted.
    line_at (float, optional): The x-position for the vertical dashed line. Default is 16.

    Returns:
    None: The function displays the histogram plot and statistics.
    """
    
    # Plot histogram of the specified column
    data[column_name].hist(bins=20, edgecolor='black')
    
    # Calculate mean and standard deviation
    mean_score = data[column_name].mean()
    std_dev_score = data[column_name].std()

    # Skewness calculation
    skewness = (np.sum((data[column_name] - mean_score) ** 3) / 
                len(data[column_name])) / (std_dev_score ** 3)

    # Kurtosis
    kurt = kurtosis(data[column_name])

    # Add a vertical red line at the specified location
    plt.axvline(x=line_at, color='red', linestyle='--', linewidth=2)

    # Add title and labels
    plt.xlabel(f'{column_name} Score', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)

    # Annotate the plot with mean, standard deviation, skewness, and kurtosis
    plt.gca().text(0.95, 0.95, 
                   f'Mean: {mean_score:.2f}\nStd Dev: {std_dev_score:.2f}\nSkewness: {skewness:.2f}\nKurtosis: {kurt:.2f}\nSize: {len(data)}', 
                   horizontalalignment='right',
                   verticalalignment='top',
                   transform=plt.gca().transAxes,
                   bbox=dict(facecolor='white', alpha=0.8, edgecolor='black'))
    
    plt.show()

## Depression Category W1A

In [ ]:
# 1 has symptoms of depression / 0 does not has symptoms of depression

df1['category_W1A'] = df1['CESD_W1A'].apply(lambda x: 0 if x < 16 else 1)
df1['category_W1A'].value_counts(normalize=True)

In [ ]:
plot_histogram_with_stats(df1, 'CESD_W1A',line_at=15)

## Depression Category W1B

In [ ]:
df1['category_W1B'] = df1['CESD_W1A'].apply(lambda x: 0 if x < 8 else 1)
df1['category_W1B'].value_counts(normalize=True)

In [ ]:
plot_histogram_with_stats(df1, 'CESD_W1B',line_at=8)

In [ ]:
# Droping ID column and the two scores A and B
df1.drop(columns=['CESD_W1A','CESD_W1B'], inplace=True)

# Wave IV

In [ ]:
df4, meta4 = pyreadstat.read_xport('REPLACE_WITH_WAVE_IV_LOCATION')

df4.head()

In [ ]:
# Clinaically Depressed

# https://addhealth.cpc.unc.edu/documentation/codebook-explorer/#/variable_collection/1077
# https://addhealth.cpc.unc.edu/documentation/codebook-explorer/#/variable_collection/1076

from matplotlib.lines import Line2D

# Filter out the entries 96, 97, and 98 from df4['H4ID6H']
clin_depre = df4[~df4['H4ID6H'].isin([96, 97, 98])]['H4ID6H']

# Define the age ranges
childhood_range = (0, 10)  # Childhood range
adolescence_range = (10, 20)
adulthood_range = (20, 100)  # Assuming age goes up to 100 for the adulthood range

# Set the figure size
plt.figure(figsize=(12, 8))

# Plot the histogram
n, bins, patches = plt.hist(clin_depre, bins=30, edgecolor='black', alpha=0.7)

# Add labels for the plot
plt.xlabel('Age of Depression (H4ID6H)')
plt.ylabel('Frequency')
plt.title('')

# Highlight the childhood area (0-12)
childhood_bin_start = (bins >= childhood_range[0]).argmax()
childhood_bin_end = (bins <= childhood_range[1]).argmin()
for i in range(childhood_bin_start, childhood_bin_end):
    patches[i].set_facecolor('green')

# Highlight the adolescence area (12-18)
adolescence_bin_start = (bins >= adolescence_range[0]).argmax()
adolescence_bin_end = (bins <= adolescence_range[1]).argmin()
for i in range(adolescence_bin_start, adolescence_bin_end):
    patches[i].set_facecolor('orange')

# Highlight the adulthood area (18-100)
adulthood_bin_start = (bins >= adulthood_range[0]).argmax()
adulthood_bin_end = len(bins) - 1  # Ensure it goes to the last bin
for i in range(adulthood_bin_start, adulthood_bin_end):
    patches[i].set_facecolor('lightblue')

# Calculate percentages of each area
total_count = n.sum()

# Calculate childhood count
childhood_count = n[childhood_bin_start:childhood_bin_end].sum()

# Calculate adolescence count
adolescence_count = n[adolescence_bin_start:adolescence_bin_end].sum()

# Calculate adulthood count
adulthood_count = n[adulthood_bin_start:adulthood_bin_end].sum()

# Calculate percentages
childhood_percentage = (childhood_count / total_count) * 100
adolescence_percentage = (adolescence_count / total_count) * 100
adulthood_percentage = (adulthood_count / total_count) * 100

# Create custom legend entries with the correct colors
childhood_legend = Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label=f'Childhood: {childhood_percentage:.1f}% ({childhood_range[0]}-{childhood_range[1]-1})')
adolescence_legend = Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10, label=f'Adolescence: {adolescence_percentage:.1f}% ({adolescence_range[0]}-{adolescence_range[1]-1})')
adulthood_legend = Line2D([0], [0], marker='o', color='w', markerfacecolor='lightblue', markersize=10, label=f'Adulthood: {adulthood_percentage:.1f}% ({adulthood_range[0]}+)')

# Add the legend to the plot
plt.legend(handles=[childhood_legend, adolescence_legend, adulthood_legend], loc='upper left', fontsize=12)

# Show the plot
plt.grid()
plt.show()

In [ ]:
adult_depre = (~df4['H4ID6H'].isin([96, 97, 98])) & (df4['H4ID6H'] >= adulthood_range[0])
adolescence_depre =  (~df4['H4ID6H'].isin([96, 97, 98])) & \
                        (df4['H4ID6H'] >= adolescence_range[0]) & \
                        (df4['H4ID6H'] <= adolescence_range[1])

# Create the new column 'adulthood_depression' based on the condition
df4['adulthood_depression'] = np.where(adult_depre, 1, 0)
df4['adolescence_depression'] = np.where(adolescence_depre, 1, 0)

print('adolescence depression:',len(adolescence_depre == 1))
print('adulthood depression:',len(adult_depre == 1))
df4.head()

In [ ]:
df4

In [ ]:
# CESD for WAVE IV
CESD4 = ["H4MH18",  # You were bothered by things that usually don't bother you.  
        "H4MH19",  #  (During the past seven days:) You could not shake off the blues, even with help from your family and your friends.  
        "H4MH20",  #  (During the past seven days:) You felt you were just as good as other people. REVERSE
        "H4MH21",  # (During the past seven days:) You had trouble keeping your mind on what you were doing. 
        "H4MH22",  # (During the past seven days:) You felt depressed.
        "H4MH23",  # (During the past seven days:) You felt that you were too tired to do things.  
        "H4MH24",  # (During the past seven days:) You felt happy.  REVERSE
        "H4MH25", # (During the past seven days:) You enjoyed life. REVERSE
        "H4MH26", # (During the past seven days:) You felt sad.  
        "H4MH27"] # (During the past seven days:) You felt that people disliked you, during the past seven days.  

df4[CESD4].head(3)

In [ ]:
# Replace 6 and 8 with NaN in the specified columns
df4[CESD4] = df4[CESD4].replace([6, 8], np.nan)

# Calculate the percentage of NaN values for each column
nan_percentage = (df4[CESD4].isna().sum())/(df4[CESD4].sum())*100

# Plotting the percentage of NaN values
plt.figure(figsize=(10, 6))
nan_percentage.sort_values().plot(kind='barh')
plt.title('Percentage of NaN Values in Selected Columns')
plt.xlabel('Percentage of NaN (%)')
plt.ylabel('Columns')
plt.axvline(x=0, color='grey', linestyle='--')  # optional: add a line at x=0
plt.show()

In [ ]:
# reversed coded: H4MH25,H4MH24,H4MH20
reversed_coded_4 = ['H4MH25','H4MH24','H4MH20']

# Apply the transformation (3 -> 0, 2 -> 1, 1 -> 2, 0 -> 3) to each of the reversed coded columns
for col in reversed_coded_4:
    df4[col] = df4[col].replace({3: 0, 2: 1, 1: 2, 0: 3})

df4[CESD4].head(3)

In [ ]:
# Sum the columns in CESD4 row-wise and store the result in a new column CESD_W4B
df4['CESD_W4B'] = df4[CESD4].sum(axis=1)

In [ ]:
plot_histogram_with_stats(df4, 'CESD_W4B',line_at=8)

In [ ]:
# WAVE IV only has W4B (10 questions)
df4['category_W4B'] = df4['CESD_W4B'].apply(lambda x: 0 if x < 10 else 1)
df4['category_W4B'].value_counts(normalize=True)

In [ ]:
print('Wave I shape: ',df1.shape, '(only the 80 predictors + the targets)')
print('Wave IV shape: ',df4.shape)
print('Genetic files shape:',gene4.shape)

count = df4['AID'].isin(df1['AID']).sum()
count_gen = gene4['AID'].isin(df1['AID']).sum()
print('Number of participants in both waves: ',count)
print('Number of participants with genetic information: ',count_gen)


# Final dataframe

If include_genetic_data is False, you should not include gene4 in the final dataframe.

If test_wave is 'Wave I', you should exclude df4 from the final dataframe.

In [ ]:
# Get the sets of AID values from each dataframe
#aid_df1 = set(df1['AID'])
#aid_df4 = set(df4['AID'])
#aid_gene4 = set(imputed_gene4['AID']) # getting the inputed version of gene4 (only the pre selected poly)

# Find the intersection of the three sets
#common_aids = aid_df1 & aid_df4 & aid_gene4

# Convert the result to a sorted list
#common_aids = sorted(common_aids)

# Print the list of AIDs
#print(common_aids)

In [ ]:
# Get column names for each dataframe
#columns_df1 = df1.columns.tolist()
#columns_df4 = df4.columns.tolist()
#columns_gene4 = imputed_gene4.columns.tolist() # imputed

In [ ]:
# Merging the DataFrames using outer join on AID
#df = pd.merge(df1, df4, on='AID', how='outer') 
#df = pd.merge(df, imputed_gene4, on='AID', how='outer') # imputed
#Outer: Ensures all rows from the three dataframes are included, with NaN for missing data.


# Droping individuals that do not have information in Wave IV or genetic
# I could not just removing NaN because there are NaN in the files
# Filter rows to keep only those with AID in common_aids
#df = df[df['AID'].isin(common_aids)]

# Reset the index for cleanliness
#df = df.reset_index(drop=True)



# Result
#print(df.shape)
#df.head()

In [ ]:
# Check the conditions in info_dict
#include_genetic_data = info_dict['include_genetic_data']
#test_wave = info_dict['test_wave']

# Get the sets of AID values from each dataframe, conditional on the settings
aid_df1 = set(df1['AID'])  # This is always included
if info_dict['test_wave'] == 'Wave I':
    # If the test_wave is 'Wave I', don't include df4
    aid_df4 = set()  # Empty set, df4 is not used
else:
    # If the test_wave is 'Wave IV', include df4
    aid_df4 = set(df4['AID'])

if info_dict['include_genetic_data']:
    # If genetic data is included, include gene4's AID values
    aid_gene4 = set(imputed_gene4['AID'])
else:
    # If genetic data is not included, don't include gene4
    aid_gene4 = set()

# Debug: print the size of each AID set
print(f"AID values in df1: {len(aid_df1)}")
print(f"AID values in df4: {len(aid_df4)}. Condition test_wave: {info_dict['test_wave']}")
print(f"AID values in gene4: {len(aid_gene4)}. Condition include_genetic_data: {info_dict['include_genetic_data']}")

# Find the intersection of the relevant AID sets
# Only calculate common_aids if we are using dataframes that are merged
common_aids = aid_df1
if aid_df4:  # Only consider df4 if it's being used
    common_aids &= aid_df4
if aid_gene4:  # Only consider gene4 if it's being used
    common_aids &= aid_gene4

# Debug: print the size of common_aids
print(f"Common AID values: {len(common_aids)}")

# Convert the result to a sorted list
common_aids = sorted(common_aids)

# Get column names for each dataframe
columns_df1 = df1.columns.tolist()
columns_df4 = df4.columns.tolist()
columns_gene4 = imputed_gene4.columns.tolist()  # imputed

# Merging DataFrames based on conditions
if info_dict['test_wave'] == 'Wave I':
    # Only merge df1 (no need to merge df4 or gene4)
    if info_dict['include_genetic_data']:
        # Merging df1 with imputed_gene4
        df = pd.merge(df1, imputed_gene4, on='AID', how='outer')
    else:
        # Only include df1 (no genetic data)
        df = df1.copy()
elif info_dict['test_wave'] == 'Wave IV':
    # Merge all three dataframes (df1, df4, and gene4)
    if info_dict['include_genetic_data']:
        # Merging df1, df4, and imputed_gene4
        df = pd.merge(df1, df4, on='AID', how='outer')
        df = pd.merge(df, imputed_gene4, on='AID', how='outer')

    else:
        # Merging df1 and df4 only (without genetic data)
        df = pd.merge(df1, df4, on='AID', how='outer')


# Debug: print the shape before filtering by common_aids
print(f"Shape before filtering: {df.shape}")

# Filter rows to keep only those with AID in common_aids
df = df[df['AID'].isin(common_aids)]

# Debug: print the shape after filtering by common_aids
print(f"Shape after filtering: {df.shape}")

# Reset the index for cleanliness
df = df.reset_index(drop=True)

# Result
print(f"Final shape: {df.shape}")



# ONLY USE THIS IF ARE DOING THE SENSTIVITY ANALYSIS - PGS MDD
## Extract the values, excluding 'FID' and 'AID' AND Major Depressive Disorder
#columns_not_drop = ["FID", "AID","Major Depressive Disorder, 2019"]
#columns_to_drop_PGS = [value for value in dictw.values() if value not in columns_not_drop]
#df = df.drop(columns=columns_to_drop_PGS)

# ONLY USE THIS IF ARE DOING THE SENSTIVITY ANALYSIS - PHYSICAL HEALTH
## Extract the values
exclude_columns_physical = ['GENERAL HEALTH','DIFFICULTY USING HANDS/FT/LIMBS','FREQ-HEADACHES','FREQ-FEELING HOT',
                           'FREQ-STOMACHACHE','FREQ-COLD SWEATS','FREQ-FEELING PHYSICALLY WEAK','FREQ-SORE THROAT/COUGH',
                            'FREQ-PAINFUL/OFTEN URINATION','FREQ-FEELING VERY SICK',
                           'FREQ-WAKE UP FEELING TIRED','FREQ-SKIN PROBLEMS','DIZZINESS','CHEST PAINS',
                            'FREQ-MUSCLE/JOINT ACHES/PAINS']

# Those were already removed: 'FREQ-VERY TIRED FOR NO REASON', 'FREQ-POOR APPETITE', 'FREQ-FEARFULNESS', ''REQ-TROUBLE RELAXING''
#df = df.drop(columns=exclude_columns_physical)

# ONLY USE THIS IF ARE DOING THE SENSTIVITY ANALYSIS - MENTAL HEALTH
## Extract the values
exclude_columns_mental = ['FREQ-INSOMNIA','NEEDED BUT NOT GET MEDICAL CARE',
                         'WEIGHT IMAGE','TO GAIN/LOSE/MAINTAIN WEIGHT','PSYCHOLOGICAL COUNSELING','HEALTH CAUSE SCHOOL ABSENCE',
                         'HEALTH CAUSE SOCIAL ABSENCE']
#df = df.drop(columns=exclude_columns_mental)


df.head()

In [ ]:
df.head()

## Data Exploration

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Compute Spearman correlation matrix
#corr_matrix = df.drop(columns=['AID', 'SEX', 'RACE', 'INCOME','category_W1A','category_W1B']).corr(method='spearman')


# Mask the upper triangle of the correlation matrix
#mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Visualize the correlation matrix using seaborn heatmap
#plt.figure(figsize=(15, 10))
#sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', fmt='.2f', linewidths=1, mask=mask) 
   
#plt.title('Spearman Correlation Matrix')
#plt.show()

# Create a set to store already printed pairs
#printed_pairs = set()

# Create a set to store columns to be dropped
#columns_to_drop = set()

# Find pairs with correlation above 0.8 and store the second variable of the pair for dropping
#for col in corr_matrix.columns:
#    for row in corr_matrix.index:
 #       # Ensure the correlation is above 0.8, not a self-correlation (i.e., row != col),
  #      # and that the pair (row, col) hasn't been printed yet
   #     if col != row and corr_matrix.loc[row, col] > 0.8:
    #        # Create a tuple (min, max) to ensure the pair is processed only once
     #       pair = tuple(sorted([row, col]))
      #      if pair not in printed_pairs:
       #         # Add the second variable in the pair (the one with the higher index) to the drop list
        #        columns_to_drop.add(pair[1])
         #       printed_pairs.add(pair)

# Drop the identified columns from the DataFrame
#df = df.drop(columns=columns_to_drop)

# Display the columns that were dropped
#print(f"Columns dropped: {columns_to_drop}")

#print(corr_matrix.shape)


### Imputation

In [ ]:
# Print the column names
print("Columns in df1:", columns_df1)
print("-----------------------------------------------------------------")
print("-----------------------------------------------------------------")
print("Columns in df4:", columns_df4)
print("-----------------------------------------------------------------")
print("-----------------------------------------------------------------")
print("Columns in gene4:", columns_gene4)

In [ ]:
# Prepare the dataset

# Convert any string representations of numbers to floats
df_imputed = df.apply(pd.to_numeric, errors='coerce')

# Save the 'ID' column
ID = df1['AID']


# Features
# List of columns to drop
columns_to_drop = df4.columns.tolist() + ['AID', 'category_W1A', 'category_W1B', 'FID'] # Add more columns if needed

# Drop the columns from df to create X
X = df_imputed.drop(columns=columns_to_drop, errors='ignore')


y_A = df_imputed['category_W1A']  # Target variable using 19 predictors using Wave I
y_B = df_imputed['category_W1B']  # Target variable using 10 predictors using Wave I
if info_dict['test_wave'] == 'Wave IV':
    y_4B = df_imputed['category_W4B'] # Target variable using 10 predictors using Wave IV

if info_dict['clinically_depressed'] == 'adulthood':
    y_4AH = df_imputed['adulthood_depression']

if info_dict['clinically_depressed'] == 'adolescence':
    y_4AC = df_imputed['adolescence_depression']

print('features:',X.shape)
X.head()

In [ ]:
## Categorical
# Start with all columns in df1 minus the specified columns
categorical_cols = df1.columns.difference(['INCOME', 'category_W1B', 'category_W1A', 'AID']).tolist()

     
# Ensure there are no duplicates
categorical_cols = list(set(categorical_cols))

# Numeric
# List of numerical columns (just 'income' initially)
numerical_cols = ['INCOME']

if info_dict['include_genetic_data']:
    # Get the columns from gene4, excluding 'FID', 'AID', 'PSANCEST', 'PSRACE'
    numerical_cols += imputed_gene4.columns.difference(['FID', 'AID', 'PSANCEST', 'PSRACE']).tolist() # imputed version

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Separate data by sex
male_data = X[X['SEX'] == 1]
female_data = X[X['SEX'] == 2]

# Get columns in X that are also in gene4
columns_in_gene4 = X.columns[X.columns.isin(imputed_gene4.columns)]

# Get columns in X that are NOT in gene4
columns_not_in_gene4 = X.columns[~X.columns.isin(imputed_gene4.columns)]

# Calculate the percentage of NaN values for columns in gene4
nan_percentage_gene4_male = male_data[columns_in_gene4].isna().mean() * 100
nan_percentage_gene4_female = female_data[columns_in_gene4].isna().mean() * 100

# Calculate the percentage of NaN values for columns NOT in gene4
nan_percentage_not_gene4_male = male_data[columns_not_in_gene4].isna().mean() * 100
nan_percentage_not_gene4_female = female_data[columns_not_in_gene4].isna().mean() * 100

# Sort and select the top 10 columns with the highest percentage of missing values
top_10_gene4_male = nan_percentage_gene4_male.sort_values(ascending=False).head(5)
top_10_gene4_female = nan_percentage_gene4_female[top_10_gene4_male.index]

top_10_not_gene4_male = nan_percentage_not_gene4_male.sort_values(ascending=False).head(5)
top_10_not_gene4_female = nan_percentage_not_gene4_female[top_10_not_gene4_male.index]

# Create a figure and set up two subplots (one on top of the other)
fig, axes = plt.subplots(2, 1, figsize=(14, 14))

# Plot for gene4 columns (Top 10) with male and female comparison
width = 0.4  # Bar width
x_gene4 = np.arange(len(top_10_gene4_male.index))  # X-axis positions

axes[0].barh(x_gene4 - width / 2, top_10_gene4_male.values, height=width, label='Male', color='skyblue', edgecolor='black')
axes[0].barh(x_gene4 + width / 2, top_10_gene4_female.values, height=width, label='Female', color='lightpink', edgecolor='black')
axes[0].set_yticks(x_gene4)
axes[0].set_yticklabels(top_10_gene4_male.index)
axes[0].set_title('Top 5 Columns with Highest Percentage of Missing Values: Genetic')
axes[0].set_xlabel('Percentage of Missing Values (%)')
axes[0].set_ylabel('Columns')
axes[0].grid(axis='x', linestyle='--')
axes[0].legend()

# Plot for non-gene4 columns (Top 10) with male and female comparison
x_not_gene4 = np.arange(len(top_10_not_gene4_male.index))  # X-axis positions

axes[1].barh(x_not_gene4 - width / 2, top_10_not_gene4_male.values, height=width, label='Male', color='skyblue', edgecolor='black')
axes[1].barh(x_not_gene4 + width / 2, top_10_not_gene4_female.values, height=width, label='Female', color='lightpink', edgecolor='black')
axes[1].set_yticks(x_not_gene4)
axes[1].set_yticklabels(top_10_not_gene4_male.index)
axes[1].set_title('Top 5 Columns with Highest Percentage of Missing Values: Environmental')
axes[1].set_xlabel('Percentage of Missing Values (%)')
axes[1].set_ylabel('Columns')
axes[1].grid(axis='x', linestyle='--')
axes[1].legend()

# Adjust the layout to avoid overlap
plt.tight_layout()
plt.show()


In [ ]:
# Impute missing values in columns specified in categorical_cols
for col in categorical_cols:
    if col in X.columns:  # Ensure the column exists in X
        X[col] = X[col].fillna(X[col].mode()[0])  # Fill NaN with the most frequent value (mode)

from sklearn.impute import KNNImputer

# Initialize KNNImputer with the desired number of neighbors (e.g., 5)
knn_imputer = KNNImputer(n_neighbors=5)

# Only use this if you are conductiing the senstitivy analysis
# Filter numerical_cols to include only those that exist in X
numerical_cols_existing = [col for col in numerical_cols if col in X.columns]

# Apply the KNN imputer on the numerical columns
#X[numerical_cols] = knn_imputer.fit_transform(X[numerical_cols])

# Only use this if you are conductiing the senstitivy analysis
# Apply the KNN imputer on the numerical columns that are present in X
X[numerical_cols_existing] = knn_imputer.fit_transform(X[numerical_cols_existing])


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize numeric data (PCAs are normalized already)
scaler = MinMaxScaler()
X['INCOME'] = scaler.fit_transform(X[['INCOME']])
X.head(3)

In [ ]:
# If comparing with clinical depression
if info_dict['clinically_depressed'] == 'adulthood' or info_dict['clinically_depressed'] == 'adolescence':
    if info_dict['clinically_depressed'] == 'adulthood':
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_4AH, test_size=0.2, random_state=42)
    else:
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_4AC, test_size=0.2, random_state=42)
    # Cross-validation setup using the combined stratification column
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # XGBoost hyperparameter tuning
    xgb_param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'n_estimators': [50, 100, 200],
        'colsample_bytree': [0.5, 0.8, 1.0],
        'subsample': [0.5, 0.8, 1.0],
        'max_features': [5, 10, 15, 20, 25]
    }
    xgb = XGBClassifier(eval_metric='logloss', random_state=42, verbosity=0)
    xgb_grid = GridSearchCV(xgb, xgb_param_grid, cv=kfold, scoring='roc_auc', n_jobs=-1, verbose=1)
    xgb_grid.fit(X_train, y_train)

    # Get the best model from the grid search
    best_xgb_model = xgb_grid.best_estimator_

    # Function to calculate Confidence Interval
    def calculate_confidence_interval(scores, confidence=0.95):
        mean_score = np.mean(scores)
        std_score = np.std(scores)
        n = len(scores)
        t_critical = stats.t.ppf((1 + confidence) / 2., n - 1)
        margin_of_error = t_critical * (std_score / np.sqrt(n))
        return mean_score - margin_of_error, mean_score + margin_of_error

    # Store results for XGBoost
    names, auc_means, auc_ci_lowers, auc_ci_uppers, accuracy_means = [], [], [], [], []
    accuracy_ci_lowers, accuracy_ci_uppers, aucs = [], [], []

    # Train and evaluate the best XGBoost model
    model_name = 'XGBoost'
    model = best_xgb_model
    

    # Train model
    model.fit(X_train, y_train)

    # Get predictions on test set
    y_prob = model.predict_proba(X_test)[:, 1]

    # Perform bootstrapping for confidence intervals
    auc, auc_mean, auc_ci_lower, auc_ci_upper, accuracy_mean, accuracy_ci_lower, accuracy_ci_upper = bootstrap_metrics(y_test, y_prob)

    # Store results
    aucs.append(auc)
    names.append(model_name)
    auc_means.append(auc_mean)
    auc_ci_lowers.append(auc_ci_lower)
    auc_ci_uppers.append(auc_ci_upper)
    accuracy_means.append(accuracy_mean)
    accuracy_ci_lowers.append(accuracy_ci_lower)
    accuracy_ci_uppers.append(accuracy_ci_upper)

    # Print results
    print(f"{model_name} Test AUC: {auc_mean:.3f} (CI: {auc_ci_lower:.3f} - {auc_ci_upper:.3f})")
    
    # Feature Importance
    importances = model.get_booster().get_score(importance_type='gain')
    
    # Convert to sorted list of (feature, importance)
    sorted_importances = sorted(importances.items(), key=lambda x: x[1], reverse=True)[:10]

    features = [item[0] for item in sorted_importances]
    gains = [item[1] for item in sorted_importances]

    # Format bars
    plt.figure(figsize=(12, 8), dpi=800)
    bars = plt.barh(range(len(features)), gains, color='#1f77b4')
    plt.yticks(range(len(features)), features)
    plt.gca().invert_yaxis()  # Highest at the top
    plt.xlabel('Gain')
    plt.title('Top 10 Feature Importances (Gain) - XGBoost')

    # Add formatted values to bars
    for i, bar in enumerate(bars):
        plt.text(bar.get_width() + max(gains)*0.01, bar.get_y() + bar.get_height()/2,
                 f'{gains[i]:.0f}', va='center')

    plt.tight_layout()
    plt.show()

    sys.exit()  # This will stop the program here.

# Simple Neural Network (classification) W1A

In [ ]:
# Function to run the 'single' model (fast model)
def run_single_model(X_train, y_train):
    # Build the neural network model
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Define the input shape
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(loss='binary_crossentropy',  # Binary cross-entropy loss for binary classification
                  optimizer=Adam(),            # Adam optimizer
                  metrics=['accuracy'])        # Track accuracy during training

    # Train the model with validation
    history = model.fit(
        X_train, y_train, 
        epochs=10, 
        batch_size=32, 
        validation_split=0.2,   # Use 20% of the training data as validation data
        verbose=1               # Print progress during training
    )
    return model, history

In [ ]:
# Function to run the 'multiple' model with hyperparameter tuning
def run_multiple_models(X_train, y_train, X_val, y_val, X_test, y_test):
    class MyBinaryClassifierHyperModel(HyperModel):
        def build(self, hp):
            model = tf.keras.Sequential()

            # Tuning the number of hidden layers
            for i in range(hp.Int('num_layers', 2, 6)):  # Number of hidden layers between 2 and 6
                model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                                                activation='relu'))

            # Add Dropout layer after each Dense layer
            model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_' + str(i), min_value=0.2, max_value=0.5, step=0.05)))

            # Output layer for binary classification
            model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Single output with sigmoid activation

            # Compile the model with tunable learning rate
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                          loss='binary_crossentropy',  # Binary classification loss
                          metrics=['accuracy'])#, 'AUC', 'Precision', 'Recall'])

            return model

    # Instantiate the hypermodel for binary classification
    hypermodel = MyBinaryClassifierHyperModel()

    # Set up the tuner with RandomSearch
    tuner = RandomSearch(
        hypermodel,
        objective='val_accuracy',  # We use validation accuracy to find the best model
        max_trials=100,  # The number of different hyperparameter combinations to try
        executions_per_trial=3,  # Number of models trained per trial (for stability)
        directory='hyperparameters-log',  # Directory where tuning results will be saved
        project_name='binary_classifier_tuning'  # Name of the tuning project
    )

       # Check shapes of the splits
    print("Training set:", X_train.shape, y_train.shape)
    print("Validation set:", X_val.shape, y_val.shape)
    print("Test set:", X_test.shape, y_test.shape)
    # Start the hyperparameter search on your dataset
    tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

    # Get the best model after tuning
    model = tuner.get_best_models(1)[0]

    # Optionally, you can evaluate the best model
    model.evaluate(X_test, y_test)

    return model

In [ ]:
def prepare_data(X, y, test_size=0.2, val_size=0.1, random_state=42, include_genetic_data=True, gene4=None,df1=None, sex_gender=None,include_environmental_data=True):
    """
    Updates X and y, handles missing values, and splits the data into train, validation, and test sets.

    Parameters:
    - X (pd.DataFrame): The feature dataset.
    - y (pd.DataFrame): The target.
    - test_size (float): Proportion of data to allocate to test and validation sets.
    - val_size (float): Proportion of total data to allocate to the validation set.
    - random_state (int): Random state for reproducibility.
    - include_genetic_data (bool): Whether to include genetic data in X.
    - gene4 (pd.DataFrame): The dataframe containing genetic information columns to exclude.
    - sex_gender (str): Filter by gender ("male" or "female").

    Returns:
    - X_train, X_val, X_test: Feature splits for train, validation, and test sets.
    - y_train, y_val, y_test: Target splits for train, validation, and test sets.
    """

    # Filter data by sex_gender if specified
    if sex_gender == "Male":
        X = X[X['SEX'] == 1]
        # Drop female-specific columns
        X = X.drop(columns=[
        "FREQ-MENSTRUAL CRAMPS",
        "FORCED SEX",
        "PREGNANCY OUTCOME",
        "Menarche, 2014",
        "Menopause, 2015",
        "SEX"], errors="ignore")  # errors="ignore" prevents errors if a column is missing

    elif sex_gender == "Female":
        X = X[X['SEX'] == 2]
        X = X.drop(columns=["SEX"], errors="ignore")  # errors="ignore" prevents errors if a column is missing

    # Ensure y aligns with X after filtering
    y = y.loc[X.index]
    
    
    # Conditionally include or remove genetic data
    if not include_genetic_data and gene4 is not None:
        # Remove all columns in gene4 except 'AID'
        genetic_columns = [col for col in gene4.columns if col != 'AID']
        X = X.drop(columns=genetic_columns, errors='ignore')

    # Conditionally include or remove enviromental data
    if not include_environmental_data and df1 is not None:
        # Remove all columns in df1 except 'AID'
        environmnetal_columns = [col for col in df1.columns if col != 'AID']
        X = X.drop(columns=environmnetal_columns, errors='ignore')

   
    # Split into training and temp (test + validation)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Split the temp set into validation and test sets
    val_test_ratio = val_size / (test_size)  # Proportion of temp data for validation
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1 - val_test_ratio, random_state=random_state)

    # Check shapes of the splits
    print("Training set:", X_train.shape, y_train.shape)
    print("Validation set:", X_val.shape, y_val.shape)
    print("Test set:", X_test.shape, y_test.shape)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
# Main function to choose between 'single' and 'multiple' models
def choose_model(X_train, y_train, X_val=None, y_val=None, X_test=None, y_test=None, model_type='single'):
    """
    Choose and run either a single model or multiple models based on the input parameters.

    Parameters:
    - X_train, y_train: Training data.
    - X_val, y_val: Validation data (optional for single model).
    - X_test, y_test: Test data (optional for single model).
    - model_type (str): Type of model to run ('single' or 'multiple').

    Returns:
    - model: The trained model(s).
    - history: Training history (for single model).
    """
    
    if model_type == 'single':
        model, history = run_single_model(X_train,y_train)
        return model, history
    elif model_type == 'multiple':
        model = run_multiple_models(X_train, y_train, X_val, y_val, X_test, y_test)
        return model
    else:
        raise ValueError("Invalid model_type. Choose either 'single' or 'multiple'.")

In [ ]:
# Check if the entire DataFrame contains any NaN values
X.isna().any().any()

In [ ]:
if info_dict['CESD_type'] == 'A':
    y = y_A
elif info_dict['CESD_type'] == 'B':
    y = y_B
else:
    ValueError("Invalid CESD_type. Choose either 'A' or 'B'.")


# Drop RACE
#X = X.drop(columns=['RACE'],errors='ignore')

X4 = X
# Filter X4 data by sex_gender if specified (if info_dict['test_wave'] == 'Wave IV')
if info_dict['test_wave'] == 'Wave IV':
    if info_dict['sex_gender'] == 'Male':
        y_4B = y_4B[X4['SEX'] == 1]
        
        X4 = X4[X4['SEX'] == 1]
        # Drop female-specific columns
        X4 = X4.drop(columns=[
            "FREQ-MENSTRUAL CRAMPS",
            "FORCED SEX",
            "PREGNANCY OUTCOME",
            "Menarche, 2014",
            "Menopause, 2015",
            "SEX"], errors="ignore")  # errors="ignore" prevents errors if a column is missing
        
    elif info_dict['sex_gender'] == 'Female':
        y_4B = y_4B[X4['SEX'] == 2]
        X4 = X4[X4['SEX'] == 2]
        X4 = X4.drop(columns=["SEX"], errors="ignore")  # errors="ignore" prevents errors if a column is missing
    
print('Target choosen:',info_dict['CESD_type'])
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(X,y,test_size=0.2, val_size=0.1, random_state=42,
                                                              include_genetic_data=info_dict['include_genetic_data'],
                                                              gene4=gene4,
                                                              df1=df1,
                                                             sex_gender = info_dict['sex_gender'],
                                                             include_environmental_data=info_dict['include_environmental_data'])

In [ ]:
# Example usage: Choose 'single' (just one model training) or 'multiple' model (turner search)
model,history = choose_model(X_train, y_train, X_val, y_val, X_test, y_test, model_type=info_dict['model_type'])

In [ ]:
# Inspect the model architecture to confirm it's correct for binary classification
model_nn = model
model.summary()

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Create subplots: 1 row, 2 columns
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Plot training & validation accuracy values on the first subplot
ax[0].plot(history.history['accuracy'], label='Train')
ax[0].plot(history.history['val_accuracy'], label='Validation')
ax[0].set_title('Model accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].set_ylim(0,1)
ax[0].legend(loc='upper left')

# Plot training & validation loss values on the second subplot
ax[1].plot(history.history['loss'], label='Train')
ax[1].plot(history.history['val_loss'], label='Validation')
ax[1].set_title('Model loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].set_ylim(0,1)
ax[1].legend(loc='upper left')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Check the model's output with a small batch of test data
output = model.predict(X_train[:5])
print("Model Output (Predicted Probabilities):")
for sublist in output:
    print([f"{sublist[0]:.6f}"])

y_pred_nn = [sublist[0] for sublist in model.predict(X_test)]

In [ ]:
title = f"Symptoms of Depression Classification (E1{info_dict['CESD_type']}  -> D1{info_dict['CESD_type']})\
          \nTarget Train and Validation: {info_dict['test_wave']} \
          \nModel Type: {info_dict['model_type']} \
          \nInclude environmental files: {info_dict['include_environmental_data']} \
          \nInclude genetic files: {info_dict['include_genetic_data']} \
          \nGender: {info_dict['sex_gender']} \
          \nTesting Size: {len(y_test)}"
if info_dict["test_wave"] == 'Wave I':
    aucs_nn ,auc_mean_nn, auc_lower_nn, auc_upper_nn, acc_mean_nn, acc_lower_nn, acc_upper_nn = plot_confusion_matrix_and_roc(y_test,
                                model,
                                X_test,
                                title,
                                info_dict=info_dict)


## Cross generation predictions

In [ ]:
# I am using the full Wave IV to predict even the ones that I use to train (I think it is fine because is another wave)
if info_dict['test_wave'] == 'Wave IV':
  title = f"Symptoms of Depression Classification (Cross Waves E1{info_dict['CESD_type']} -> D4B) \
          \nTarget Train and Validation: {info_dict['test_wave']} \
        \nModel Type: {info_dict['model_type']}\nInclude genetic files: {info_dict['include_genetic_data']} \
        \n Gender: {info_dict['sex_gender']}\
        \nTesting Size: {len(y_4B)}" 

if info_dict["test_wave"] == 'Wave IV':
    aucs_nn ,auc_mean_nn, auc_lower_nn, auc_upper_nn, acc_mean_nn, acc_lower_nn, acc_upper_nn = plot_confusion_matrix_and_roc(y_4B,
                                model,
                                X4,
                                title,
                                info_dict=info_dict)

## Feature Importance

## Bewwsarm/sumary plot
- Features on the Y-axis:
    - Each dot represents a feature in your dataset. 
    - The Y-axis will list your input features.
- SHAP values on the X-axis:
    - SHAP values represent the contribution of each feature to the model’s prediction for a particular instance. 
        - Positive SHAP values push the prediction higher (towards 1, since it's a binary classification with sigmoid activation) 
        - Negative SHAP values push it lower (towards 0).
    - A feature that has a large positive SHAP value means it contributed significantly to the model predicting a positive class (1) high depression symptoms. 
    - A feature with a large negative SHAP value means it pushed the model's prediction towards the negative class (0) lower depression symptoms.
- Dot Density and Spread:
    - The distribution of dots on the plot will show the variation of the feature's SHAP values across all instances in your dataset.
    - Features with a wider spread and more varied SHAP values are more influential across different instances, while features with a narrow spread might be less impactful overall.
    - If you see a lot of dots concentrated far to the right (positive SHAP values), it means that feature usually contributes positively towards predicting the positive class.

- Coloring:

    - Typically, the color of each dot corresponds to the feature's value (or another related aspect). For example, red might indicate higher values of a feature, while blue might represent lower values. This color gradient helps you see how the value of the feature correlates with the direction of the SHAP value (whether it’s pushing the model prediction higher or lower).

In [ ]:
import shap
import random

# Set the seed for reproducibility
random.seed(42)
np.random.seed(42)

# Use a small subset of data to speed up SHAP computation
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test[:500])  # Compute SHAP values for 100 samples

# Plot feature importance
shap.summary_plot(shap_values, max_display=10)

In [ ]:
shap.plots.waterfall(shap_values[101], max_display=10)

In [ ]:
bronfenbrenner_dict = {
    # Individual (Personal Attributes & Health)
    'SEX': 'Individual',
    'RACE': 'Individual',
    'GENERAL HEALTH': 'Individual',
    'DIFFICULTY USING HANDS/FT/LIMBS': 'Individual',
    'FREQ-HEADACHES': 'Individual',
    'FREQ-FEELING HOT': 'Individual',
    'FREQ-STOMACHACHE': 'Individual',
    'FREQ-COLD SWEATS': 'Individual',
    'FREQ-FEELING PHYSICALLY WEAK': 'Individual',
    'FREQ-SORE THROAT/COUGH': 'Individual',
    'FREQ-VERY TIRED FOR NO REASON': 'Individual',
    'painful or very frequent urination (or peeing)': 'Individual',
    'feeling really sick': 'Individual',
    'FREQ-WAKE UP FEELING TIRED': 'Individual',
    'skin problems, such as itching or pimples': 'Individual',
    'dizziness': 'Individual',
    'chest pains': 'Individual',
    'aches, pains, or soreness in your muscles or joints': 'Individual',
    'FREQ-MENSTRUAL CRAMPS': 'Individual',
    'FREQ-POOR APPETITE': 'Individual',
    'FREQ-INSOMNIA': 'Individual',
    'FREQ-TROUBLE RELAXING': 'Individual',
    'FREQ-FEARFULNESS': 'Individual',
    'WEIGHT IMAGE': 'Individual',
    'TO GAIN/LOSE/MAINTAIN WEIGHT': 'Individual',
    'LIKE SELF AS ARE': 'Individual',
    'DO EVERYTHING JUST RIGHT': 'Individual',
    'FEEL SOCIALLY ACCEPTED': 'Individual',
    'FEEL LOVED AND WANTED': 'Individual',

    # Microsystem (Immediate Relationships & Environments)
    'CLOSE TO MOM': 'Microsystem',
    'MOM-HOW MUCH DOES SHE CARE': 'Microsystem',
    'CLOSE TO DAD': 'Microsystem',
    'DAD-HOW MUCH DOES HE CARE': 'Microsystem',
    'MOM-WARM AND LOVING': 'Microsystem',
    'MOM-GOOD COMMUNICATION': 'Microsystem',
    'MOM-GOOD RELATIONSHIP': 'Microsystem',
    'DAD-GOOD COMMUNICATION': 'Microsystem',
    'DAD-GOOD RELATIONSHIP': 'Microsystem',
    'FAMILY UNDERSTAND YOU': 'Microsystem',
    'FAMILY PAYS ATTENTION TO YOU': 'Microsystem',
    'FAMILY HAS FUN TOGETHER': 'Microsystem',
    'PARENTS CARE ABOUT YOU': 'Microsystem',
    'WANT TO LEAVE HOME': 'Microsystem',
    'HANG OUT WITH FRIENDS': 'Microsystem',
    'FRIENDS CARE ABOUT YOU': 'Microsystem',
    'PAST YR-FRIENDS ATTEMPT SUICIDE': 'Microsystem',
    'PAST YR-FRIENDS SUCCEED': 'Microsystem',
    'You feel close to people at your school.': 'Microsystem',
    'FEEL PART OF YOUR SCHOOL': 'Microsystem',
    'STUDENTS AT SCHOOL PREJUDICED': 'Microsystem',
    'HAPPY AT YOUR SCHOOL': 'Microsystem',
    'The teachers at your school treat students fairly.': 'Microsystem',
    'FEEL SAFE IN YOUR SCHOOL': 'Microsystem',
    'TEACHERS CARE ABOUT YOU': 'Microsystem',
    'TROUBLE-GETTING ALONG TEACHERS': 'Microsystem',
    'paying attention in school?': 'Microsystem',
    'TROUBLE-GETTING HOMEWORK DONE': 'Microsystem',
    'TROUBLE-WITH OTHER STUDENTS': 'Microsystem',

    # Mesosystem (Interactions Between Microsystems)
    'FAMILY UNDERSTAND YOU': 'Mesosystem',
    'FAMILY PAYS ATTENTION TO YOU': 'Mesosystem',
    'TEACHERS CARE ABOUT YOU': 'Mesosystem',
    'TROUBLE-WITH OTHER STUDENTS': 'Mesosystem',
    'PAST YR-FRIENDS ATTEMPT SUICIDE': 'Mesosystem',

    # Exosystem (Indirect Environmental Influences)
    'INCOME': 'Exosystem',
    'ENOUGH MONEY FOR BILLS': 'Exosystem',
    'MEDICAL CARE FOR FAMILY': 'Exosystem',
    'NEEDED BUT NOT GET MEDICAL CARE': 'Exosystem',
    'WHY-COULD NOT PAY': 'Exosystem',
    'FEEL SAFE IN NBORHOOD?': 'Exosystem',
    'HOW HAPPY LIVING IN NBORHOOD': 'Exosystem',
    'SAW SHOOTING/STABBING OF PERSON': 'Exosystem',
    'HAD KNIFE/GUN PULLED ON YOU': 'Exosystem',
    'SOMEONE SHOT YOU': 'Exosystem',
    'SOMEONE STABBED YOU': 'Exosystem',
    'GOT INTO A PHYSICAL FIGHT': 'Exosystem',
    'WERE JUMPED': 'Exosystem',
    'PAST YR-FAMILY ATTEMPT SUICIDE': 'Exosystem',
    'PAST YR-FAMILY SUCCEED': 'Exosystem',

    # Macrosystem (Cultural & Societal Influences)
    'FORCED SEX': 'Macrosystem',
    'SEX FOR DRUGS OR MONEY': 'Macrosystem',

    # Chronosystem (Life Events & Changes Over Time)
    'PREG1 PREGNANCY OUTCOME': 'Chronosystem',
    'HEALTH CAUSE SCHOOL ABSENCE': 'Chronosystem',
    'HEALTH CAUSE SOCIAL ABSENCE': 'Chronosystem',
    'PAST YR-FRIENDS ATTEMPT SUICIDE': 'Chronosystem',
    'PAST YR-FRIENDS SUCCEED': 'Chronosystem',
    'PAST YR-FAMILY ATTEMPT SUICIDE': 'Chronosystem',
    'PAST YR-FAMILY SUCCEED': 'Chronosystem'
}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming shap_values is your SHAP Explanation object
shap_values_array = shap_values.values  # Extract SHAP values

# Compute the mean absolute SHAP values for each feature
mean_abs_shap_values = np.abs(shap_values_array).mean(axis=0)

# Get the feature names (columns of X_train)
features = X_train.columns  # Assuming X_train is your DataFrame

# Sort the mean absolute SHAP values in descending order
sorted_idx = np.argsort(mean_abs_shap_values)[::-1]  # Indices that would sort the values in descending order
sorted_values = mean_abs_shap_values[sorted_idx]
sorted_features = features[sorted_idx]

# Select only the top 10 features
top_10_values = sorted_values[:10]
top_10_features = sorted_features[:10]

# Define color mapping for Bronfenbrenner categories
category_colors = {
    'Individual': 'lightblue',
    'Microsystem': 'lightgreen',
    'Mesosystem': 'mediumseagreen',
    'Exosystem': 'purple',
    'Macrosystem': 'orange',
    'Chronosystem': 'red'
}

# Assuming 'bronfenbrenner_dict' is available and maps feature names to categories
# Determine colors based on df1 membership and Bronfenbrenner category
colors = []
labels = []

for feature in top_10_features:
    if feature in bronfenbrenner_dict:
        # If the feature is in bronfenbrenner_dict, use the corresponding color
        category = bronfenbrenner_dict.get(feature, 'gold')
        colors.append(category_colors.get(category, 'gold'))  # Default to 'gold' if category not found
        labels.append(category)  # Use the category as label
    else:
        # If the feature is not in the bronfenbrenner_dict, color it gold and label it "Polygenic Scores"
        colors.append('gold')
        labels.append('Polygenic Scores')

# Create the horizontal bar plot for top 10 with color mapping
plt.figure(figsize=(10, 8))
bars = plt.barh(range(len(top_10_values)), top_10_values, color=colors, edgecolor='black')

# Label the axes and set the title
plt.ylabel('Feature')
plt.xlabel('Mean Absolute SHAP Value')
plt.title('Top 10 Features by Mean Absolute SHAP Value')

# Set the feature names as y-tick labels (larger values are at the top)
plt.yticks(range(len(top_10_values)), top_10_features)  # Use top_10_features as y-tick labels

# Optionally, add legend to indicate categories (based on the colors)
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in category_colors.values()] + [plt.Rectangle((0, 0), 1, 1, color='gold')]
labels = list(category_colors.keys()) + ['Polygenic Scores']

# Move the legend to the top-right corner inside the plot
plt.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.95, 0.95))

# Show the plot
plt.show()

In [ ]:
X

# Decision Tree / XGBoost / SVM

In [ ]:
# Suppress XGBoost warnings
warnings.simplefilter(action='ignore', category=UserWarning)
os.environ['XGBOOST_ENABLE_WARNINGS'] = '0'


# Cross-validation setup using the combined stratification column
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# XGBoost hyperparameter tuning
xgb_param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'subsample': [0.5, 0.8, 1.0],
    'max_features': [5, 10, 15, 20, 25]
}
xgb = XGBClassifier(eval_metric='logloss', random_state=42, verbosity=0)
xgb_grid = GridSearchCV(xgb, xgb_param_grid, cv=kfold, scoring='roc_auc', n_jobs=-1, verbose=1)
xgb_grid.fit(X_train, y_train)

# Initialize models with tuned XGBoost
models = {
    'Decision Tree': DecisionTreeClassifier(min_samples_leaf=150, min_samples_split=75, random_state=42),
    'SVM (Linear)': SVC(kernel='linear', probability=True, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=42),
    'XGBoost': xgb_grid.best_estimator_
}

# Function to calculate Confidence Interval
def calculate_confidence_interval(scores, confidence=0.95):
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    n = len(scores)
    t_critical = stats.t.ppf((1 + confidence) / 2., n - 1)
    margin_of_error = t_critical * (std_score / np.sqrt(n))
    return mean_score - margin_of_error, mean_score + margin_of_error

# Store results
names, auc_means, auc_ci_lowers, auc_ci_uppers, accuracy_means = [], [], [], [], []
accuracy_ci_lowers, accuracy_ci_uppers, aucs, y_pred_list = [], [], [], []

# If we are analysing Wave IV, the test set is all of the values in Wave IV, because we trained in Wave I.
if info_dict["test_wave"] == 'Wave IV':
    X_test = X4
    y_test = y_4B
 
for name, model in tqdm(models.items(), desc="Training models"):
    model.fit(X_train, y_train)  # Train model

    
    # Get predictions on test set
    y_prob = model.predict_proba(X_test)[:, 1]

    # Perform bootstrapping for confidence intervals
    auc, auc_mean, auc_ci_lower, auc_ci_upper, accuracy_mean, accuracy_ci_lower, accuracy_ci_upper = bootstrap_metrics(y_test, y_prob)
    
    # Store results
    aucs.append(auc)
    names.append(name)
    auc_means.append(auc_mean)
    auc_ci_lowers.append(auc_ci_lower)
    auc_ci_uppers.append(auc_ci_upper)
    accuracy_means.append(accuracy_mean)
    accuracy_ci_lowers.append(accuracy_ci_lower)
    accuracy_ci_uppers.append(accuracy_ci_upper)
    y_pred_list.append(y_prob.tolist())
    print(y_prob)
    
    # Print results
    print(f"{name} Test AUC: {auc_mean:.3f} (CI: {auc_ci_lower:.3f} - {auc_ci_upper:.3f})")
    print(f"{name} Test Accuracy: {accuracy_mean:.3f} (CI: {accuracy_ci_lower:.3f} - {accuracy_ci_upper:.3f})\n")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Plot the decision tree with larger figure size
plt.figure(figsize=(120, 50), dpi=200)  # Increase figure size
plot_tree(models['Decision Tree'], 
          filled=True, 
          feature_names=X_train.columns, 
          class_names=['0', '1'], 
          fontsize=10,   # Font size
          rounded=True,  # Rounded corners
          proportion=True)  # Proportional sizes of the tree nodes

plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression

# Train the Logistic Regression model
model_logistic_regression = LogisticRegression()
model_logistic_regression.fit(X_train, y_train)

# Make predictions and get predicted probabilities
y_test_pred_lr = model_logistic_regression.predict(X_test)
y_test_prob_lr = model_logistic_regression.predict_proba(X_test)[:, 1]  # Probabilities for AUC calculation

auc_lr, auc_mean_lr, auc_ci_lower_lr, auc_ci_upper_lr, accuracy_mean_lr, accuracy_ci_lower_lr, accuracy_ci_upper_lr = bootstrap_metrics(y_test, y_test_prob_lr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Check if "Logistic Regression" is not already in the names list
if "Logistic Regression" not in names:
    models = {"Logistic Regression": model_logistic_regression, **models}
    names.insert(0, "Logistic Regression")  
    aucs.insert(0, auc_lr) 
    auc_means.insert(0, auc_mean_lr)  
    auc_ci_lowers.insert(0, auc_ci_lower_lr)  
    auc_ci_uppers.insert(0, auc_ci_upper_lr)  
    accuracy_means.insert(0, accuracy_mean_lr)  
    accuracy_ci_lowers.insert(0, accuracy_ci_lower_lr)  
    accuracy_ci_uppers.insert(0, accuracy_ci_upper_lr)
    y_pred_list.insert(0, y_test_prob_lr)


# Check if "Neural Network" is not already in the names list
if "Neural Network" not in names:
    models["Neural Network"] = model_nn
    names.append("Neural Network")
    aucs.append(aucs_nn)
    auc_means.append(auc_mean_nn)
    auc_ci_lowers.append(auc_lower_nn)
    auc_ci_uppers.append(auc_upper_nn)
    accuracy_means.append(acc_mean_nn)
    accuracy_ci_lowers.append(acc_lower_nn)
    accuracy_ci_uppers.append(acc_upper_nn)
    y_pred_list.append(y_pred_nn)

# Compute error bars
auc_errors = [(auc_means[i] - auc_ci_lowers[i], auc_ci_uppers[i] - auc_means[i]) for i in range(len(names))]
accuracy_errors = [(accuracy_means[i] - accuracy_ci_lowers[i], accuracy_ci_uppers[i] - accuracy_means[i]) for i in range(len(names))]

auc_errors = np.array(auc_errors).T
accuracy_errors = np.array(accuracy_errors).T

# Plot side-by-side figures
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns
x_pos = np.arange(len(names))  # Position of bars

# Accuracy plot (left)
bars_acc = axes[0].bar(x_pos, accuracy_means, yerr=accuracy_errors, capsize=5, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title("Model Accuracy")
axes[0].set_ylabel("Accuracy")
axes[0].set_xticks(x_pos)  # Set tick positions
axes[0].set_xticklabels(names, rotation=45, ha='right')  # Set tick labels
axes[0].set_ylim(0.725, .875)

# Add the values on top of each bar in the accuracy plot
for i, bar in enumerate(bars_acc):
    yval = bar.get_height()
    error_top = accuracy_errors[1, i]  # Get the upper error bar for this bar
    axes[0].text(bar.get_x() + bar.get_width() / 2, yval + error_top  - 0.05,  # Slight offset above error bar
                 f'{yval:.3f}', ha='center', va='bottom', fontsize=10)

# AUC plot (right)
bars_auc = axes[1].bar(x_pos, auc_means, yerr=auc_errors, capsize=5, color='steelblue', edgecolor='black', alpha=0.7)
axes[1].set_title("Model AUC")
axes[1].set_ylabel("AUC")
axes[1].set_xticks(x_pos)  # Set tick positions
axes[1].set_xticklabels(names, rotation=45, ha='right')  # Set tick labels
axes[1].set_ylim(0.7, .9)

# Add the values on top of each bar in the plot
for i, bar in enumerate(bars_auc):
    yval = bar.get_height()
    error_top = auc_errors[1, i]  # Get the upper error bar for this bar
    axes[1].text(bar.get_x() + bar.get_width() / 2, yval + error_top - 0.06,  # Slight offset above error bar
                 f'{yval:.3f}', ha='center', va='bottom', fontsize=10)

# Adjust layout
plt.tight_layout()
plt.show()

# Print AUC error bars on screen
print("AUC Confidence Intervals:")
for i, name in enumerate(names):
    mean = auc_means[i]
    lower = auc_ci_lowers[i]
    upper = auc_ci_uppers[i]
    sd = auc_stds[i] if 'auc_stds' in locals() else 'N/A'
    n = auc_ns[i] if 'auc_ns' in locals() else 'N/A'
    print(f"{name}: mean = {mean:.3f}, SD = {sd if isinstance(sd, str) else f'{sd:.3f}'}, N = {n}")
    print(f"    CI = [{lower:.3f}, {upper:.3f}]")
    


In [ ]:
# Print Accuracy and AUC with Confidence Intervals

print('Test Wave:',info_dict['test_wave'])
print('Sex:',info_dict['sex_gender']) # None is both
print('Include environmental?',info_dict['include_environmental_data'])
print('Include genetic?',info_dict["include_genetic_data"])
print("Model Performance Metrics:")
for i, name in enumerate(names):
    print(f"{name}:")
    print(f"  Accuracy: {accuracy_means[i]:.3f} "
          f"(95% CI: {accuracy_ci_lowers[i]:.3f} - {accuracy_ci_uppers[i]:.3f})")
    print(f"  AUC:      {auc_means[i]:.3f} "
          f"(95% CI: {auc_ci_lowers[i]:.3f} - {auc_ci_uppers[i]:.3f})\n")

### The order is Logistic Regression [0], Decision Tree [1], SVM (Linear) [2], SVM (RBF) [3], XGBoost [4]  and Neural Network [5]

In [ ]:
# Pairwise t-test

from itertools import combinations
from scipy.stats import ttest_rel, wilcoxon
import numpy as np

# Create dictionary using enumerate to map names to their indices
model_indices = {name: index for index, name in enumerate(names)}

def compare_distributions(auc_scores_1, auc_scores_2, dist_name_1="Distribution 1", dist_name_2="Distribution 2", alpha=0.05):
    """
    Compares two distributions and returns whether they are statistically different.
    
    Returns:
    (dist_name_1, dist_name_2, p_ttest, p_wilcoxon)
    """
    if len(auc_scores_1) != len(auc_scores_2):
        raise ValueError("The arrays must have the same length for paired tests.")
    
    t_stat, p_value = stats.ttest_ind(auc_scores_1, auc_scores_2,equal_var=False)
    #w_stat, p_wilcoxon = wilcoxon(auc_scores_1, auc_scores_2)
    
    return dist_name_1, dist_name_2, t_stat, p_value#, p_wilcoxon

alpha = 0.05

for name1, name2 in combinations(names, 2):
    aucs_1 = aucs[model_indices[name1]]
    aucs_2 = aucs[model_indices[name2]]

    dist1, dist2, t_stat, p_value = compare_distributions(aucs_1, aucs_2, dist_name_1=name1, dist_name_2=name2, alpha=alpha)

    #if p_ttest >= alpha and p_wilcoxon >= alpha:
    #    print(f"No significant difference between {dist1} and {dist2}:")
    #    print(f"  Paired t-test p = {p_ttest:.5f}, Wilcoxon p = {p_wilcoxon:.5f}")
    print(f"{dist1} vs. {dist2} (t = {abs(t_stat):.2f}, p = {p_value:.3f})")


In [ ]:
# Rossmann does not allow to download from the MLstatkit
"""
from MLstatkit import Delong_test 

#models = [
    "Logistic Regression", "Decision Tree", "SVM (Linear)", 
    "SVM (RBF)", "XGBoost", "Neural Network"
]

# Iterate over all pairs of models (i.e., rows 0-1, 0-2, ..., 4-5)
for i in range(len(models)):
    for j in range(i+1, len(models)):  # Avoid comparing a model to itself
        # Get the rows from the DataFrame for the models i and j
        model_1 = df.iloc[i].tolist()
        model_2 = df.iloc[j].tolist()

        # Perform the DeLong test
        z, p = Delong_test(y_test, model_1, model_2, return_ci=False, return_auc=False, verbose=0)

        # Print the results for this pair of models
        print(f"Comparison: {models[i]} vs {models[j]}")
        print(f"z = {z:.6f}, p = {p:.3e}")
        print("-" * 50)  # For separation between comparisons
        
"""""

In [ ]:
# Anova

from itertools import combinations
from scipy import stats
import numpy as np

# Create dictionary using enumerate to map names to their indices
model_indices = {name: index for index, name in enumerate(names)}

def compare_distributions_anova(auc_scores_list, model_names, alpha=0.05):
    """
    Performs one-way ANOVA to compare multiple distributions and checks if there's a significant difference.
    
    Returns:
    (anova_statistic, p_value)
    """
    # Perform ANOVA test
    f_stat, p_value = stats.f_oneway(*auc_scores_list)
    
    return f_stat, p_value

alpha = 0.05

# Prepare list of AUC scores for all models
auc_scores_list = [aucs[model_indices[name]] for name in names]

# Perform ANOVA across all distributions
f_stat, p_value = compare_distributions_anova(auc_scores_list, names, alpha)

if p_value < alpha:
    print(f"Significant difference found between distributions (F-statistic = {f_stat:.2f}, p = {p_value:.3f})")
else:
    print(f"No significant difference between distributions (F-statistic = {f_stat:.2f}, p = {p_value:.3f})")
   

In [ ]:
namee = 'XGBoost'
print(namee)
print('mean =',round(aucs[model_indices[namee]].mean(),4))
print('std =',round(aucs[model_indices[namee]].std(),4))
print('SE =',round(stats.sem(aucs[model_indices[namee]]),4))
print('N =',len(aucs[model_indices[namee]]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Reversing the dictionary to map index to model name
index_to_model = {v: k for k, v in model_indices.items()}

def get_model_by_index(index):
    return index_to_model.get(index, "Model not found")

plt.figure(figsize=(10, 6))

tab10 = plt.get_cmap('tab10')
color_pallet = {i: tab10(i) for i in range(6)}


all_counts = []
all_means = []

for i in range(len(model_indices)):
    data = aucs[i]

    counts, bins, patches = plt.hist(
        data,
        bins=20,
        histtype='step',
        linewidth=2,
        edgecolor=color_pallet[i],
        label=get_model_by_index(i)
    )
    all_counts.append(max(counts))

    mean_auc = np.mean(data)
    all_means.append(mean_auc)
    plt.axvline(
        mean_auc,
        color=color_pallet[i],
        linestyle='--',
        linewidth=1
    )

max_y = max(all_counts)
arrow_y_base = max_y * 0.05
arrow_y_spacing = max_y * 0.08

text_y_base = -max_y * 0.05  # Base height for text below x-axis
text_y_step = -max_y * 0.03  # Step to vary height and reduce overlap

for i in range(len(model_indices)):
    data = aucs[i]
    ci_low, ci_high = np.percentile(data, [2.5, 97.5])
    mean_auc = all_means[i]
    color = color_pallet[i]

    # Draw CI arrows
    arrow_y = arrow_y_base + i * arrow_y_spacing
    plt.annotate(
        '', xy=(ci_high, arrow_y), xytext=(ci_low, arrow_y),
        arrowprops=dict(
            arrowstyle='<->',
            color=color,
            linewidth=1.
        )
    )

    text_y = text_y_base + i * text_y_step  # Vary y to reduce stacking

    text = f"{mean_auc:.3f} [{ci_low:.3f}–{ci_high:.3f}]"
    plt.text(
        mean_auc, text_y,
        text,
        color=color,
        fontsize=9,
        ha='center',
        va='top',
        rotation=0
    )

plt.xlabel('AUC Score')
plt.ylabel('Frequency')
plt.title('Bootstrapping Distributions')
plt.legend()
plt.tight_layout()
plt.subplots_adjust(bottom=0.25)  # More space for labels
plt.show()


In [ ]:
# Create a dictionary for the DataFrame
models_dict = {
    "Model":models.values,
    "Name": names,
    "AUC": aucs,
    "AUC Mean": auc_means,
    "AUC CI Lower": auc_ci_lowers,
    "AUC CI Upper": auc_ci_uppers,
    "Accuracy Mean": accuracy_means,
    "Accuracy CI Lower": accuracy_ci_lowers,
    "Accuracy CI Upper": accuracy_ci_uppers,
    "Prob y": y_pred_list
}

# Create DataFrame
models_df = pd.DataFrame(models_dict)

# Display the DataFrame
models_df

In [ ]:
models_df['AUC'].iloc[0].shape

# Rank Concordance

In [ ]:
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
import numpy as np

# Create a function to extract feature importance for each model
def get_feature_importance(model, X_train, y_train, model_name):
    if hasattr(model, 'feature_importances_'):
        # Directly get feature importances (e.g., DecisionTree, XGBoost)
        return model.feature_importances_
    
    elif isinstance(model, SVC) and model.kernel == 'linear':
        # For SVM (Linear), use the absolute value of the coefficients
        return np.abs(model.coef_[0])

    elif isinstance(model, SVC) and model.kernel == 'rbf':
        # Perform permutation importance and return only importances_mean
        result = permutation_importance(model, X_train, y_train, n_jobs=-1, max_samples=0.02, n_repeats=2)
        return result.importances_mean  # Extract the importance means

    elif isinstance(model, LogisticRegression):
        # For Logistic Regression, use the absolute value of the coefficients
        return np.abs(model.coef_[0])

    elif model_name == 'Neural Network':
        # For Neural Network, use SHAP values
        return np.abs(shap_values.values).mean(axis=0)  # Mean of absolute SHAP values

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

# Function to plot feature importances of multiple models using subplots
def plot_feature_importances(models, X_train, y_train, top_n=10):
    importances = {}

    # Check if models are correctly passed
    if not models:
        print("No models provided.")
        return

    # Create a subplot grid with an optimal number of rows and columns
    model_names = list(models.keys())
    num_models = len(model_names)

    # Calculate the number of rows and columns based on the number of models
    num_rows = math.ceil(np.sqrt(num_models))  # Approximate a square grid
    num_cols = math.ceil(num_models / num_rows)

    # Create subplots with calculated rows and columns
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(5 * num_cols, 5 * num_rows))

    # Flatten the axes array for easy indexing (in case it's a multi-dimensional grid)
    axes = axes.flatten()

    # Loop through each model and plot its feature importances
    for i, (model_name, model) in enumerate(models.items()):
        # Get the feature importances
        feature_importance = get_feature_importance(model, X_train, y_train, model_name)

        # Rank the features based on importance (most to least important)
        ranked_indices = np.argsort(feature_importance)[::-1]
        sorted_importance = feature_importance[ranked_indices]
        sorted_features = np.array(X_train.columns)[ranked_indices]

        # Only take the top_n features
        top_features = sorted_features[:top_n]
        top_importance = sorted_importance[:top_n]

        # Plot the top feature importances
        axes[i].barh(top_features, top_importance, color='lightblue')
        axes[i].set_title(f'{model_name} Feature Importances')
        axes[i].set_xlabel('Importance')
        axes[i].set_ylabel('Feature')

        # Adjust the tick label sizes
        axes[i].tick_params(axis='y', labelsize=10)
        axes[i].tick_params(axis='x', labelsize=8)

    # Hide any unused axes (if the grid size exceeds the number of models)
    for j in range(num_models, len(axes)):
        axes[j].axis('off')

    # Adjust layout to avoid overlap
    plt.tight_layout()
    plt.show()


plot_feature_importances(models, X_train, y_train, top_n=10)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr

# Function to plot pairwise rank concordance
def plot_pairwise_rank_concordance(models, X_train, y_train, top_n=10):
    importances = {}
    rankings = {}

    # Check if models are correctly passed
    if not models:
        print("No models provided.")
        return

    # Get the list of model names
    model_names = list(models.keys())
    print("Model names:", model_names)  # Debugging line to check the list of model names

    # Check if there are enough models
    if len(model_names) < 2:
        print("Not enough models for pairwise comparison.")
        return

    # Store feature importances and rankings for each model
    for model_name, model in models.items():
        print(f"Processing model: {model_name}")

        # Get the feature importances
        feature_importance = get_feature_importance(model, X_train, y_train, model_name)

        # Rank the features based on importance (most to least important)
        ranked_indices = np.argsort(feature_importance)[::-1]

        # Save feature importances and their rankings
        importances[model_name] = feature_importance
        rankings[model_name] = ranked_indices
    
    # Calculate the number of subplots (pairwise comparisons)
    num_comparisons = len(model_names) * (len(model_names) - 1) // 2
    #cols = int(np.ceil(np.sqrt(num_comparisons)))  # Make grid as square as possible
    #rows = int(np.ceil(num_comparisons / cols))
    
    # Set the number of columns to 3
    cols = 3

    # Calculate the number of rows needed for the comparisons
    rows = int(np.ceil(num_comparisons / cols))

    # Create subplots
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
    axes = axes.flatten()  # Flatten axes for easy indexing

    # Now compare the rank concordance (Spearman's rank correlation) between models
    plot_index = 0
    for i in range(len(model_names)):
        for j in range(i + 1, len(model_names)):
            model1 = model_names[i]
            model2 = model_names[j]

            # Calculate Spearman's rank correlation between the rankings
            rank_corr, _ = spearmanr(rankings[model1], rankings[model2])
            print(f"Spearman's rank correlation between {model1} and {model2}: {rank_corr:.4f}")

            ax = axes[plot_index]  # Get the next subplot axis

            # Plot all features (light grey for the rest)
            ax.scatter(importances[model1], importances[model2], c='lightgrey', edgecolors='k', alpha=0.7, s=300)

            # Highlight the top `n` common features
            common_top_features = list(set(rankings[model1][:top_n]) | set(rankings[model2][:top_n]))
            for idx, k in enumerate(common_top_features):
                ax.scatter(importances[model1][k], importances[model2][k], c=[plt.cm.tab20(idx)], s=300, label=X_train.columns[k], edgecolors='k', alpha=0.7)

            # Add labels and title
            ax.set_xlabel(f'{model1} Feature Importance')
            ax.set_ylabel(f'{model2} Feature Importance')
            ax.set_title(f"Rank Concordance: {model1} vs {model2}\nSpearman's Corr: {rank_corr:.4f}")

            # Legend and grid
            ax.legend(title="Top Features", fontsize=8, markerscale=0.5, handlelength=2)
            ax.grid(True)

            plot_index += 1

    # Remove any unused axes
    for i in range(plot_index, len(axes)):
        axes[i].axis('off')

    # Adjust layout to avoid overlap
    plt.tight_layout()
    plt.show()

# Plot
plot_pairwise_rank_concordance(models, X_train, y_train, top_n=5)


# Feature Importance Best model (XGBoost)

In [ ]:
# Get feature importance from the model
booster = models['XGBoost'].get_booster()

# Get feature importance scores for each importance type
importance_weight = booster.get_score(importance_type='weight')
importance_gain = booster.get_score(importance_type='gain')
importance_cover = booster.get_score(importance_type='cover')

# Round the importance values to integers
importance_weight = {k: round(v) for k, v in importance_weight.items()}
importance_gain = {k: round(v) for k, v in importance_gain.items()}
importance_cover = {k: round(v) for k, v in importance_cover.items()}

# Sort the features by importance for each type and get the top 10
top_10_weight = sorted(importance_weight.items(), key=lambda x: x[1], reverse=True)[:10]
top_10_gain = sorted(importance_gain.items(), key=lambda x: x[1], reverse=True)[:10]
top_10_cover = sorted(importance_cover.items(), key=lambda x: x[1], reverse=True)[:10]

# Separate feature names and importance values for plotting
features_weight, values_weight = zip(*top_10_weight)
features_gain, values_gain = zip(*top_10_gain)
features_cover, values_cover = zip(*top_10_cover)

# Set up subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot feature importance based on 'weight' (F-score)
axes[0].barh(features_weight, values_weight, align="center")
axes[0].set_title('Feature Importance - Weight (F-score)')
axes[0].set_xlabel('Importance')
axes[0].set_ylabel('Features')

# Plot feature importance based on 'gain'
axes[1].barh(features_gain, values_gain, align="center")
axes[1].set_title('Feature Importance - Gain')
axes[1].set_xlabel('Importance')
axes[1].set_ylabel('Features')

# Plot feature importance based on 'cover'
axes[2].barh(features_cover, values_cover, align="center")
axes[2].set_title('Feature Importance - Cover')
axes[2].set_xlabel('Importance')
axes[2].set_ylabel('Features')

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
dict(zip(features_gain, values_gain))

In [ ]:
# Plotting the feature importance
#plt.figure(figsize=(12, 8), dpi=800)


# Plot using 'gain' as the importance type (which corresponds to Gini index)#
#xgb.plot_importance(models['XGBoost'], importance_type='gain', max_num_features=10, height=0.8,show_values=True,values_format ='{v:.0f}')

#plt.savefig("feature_importance_xgboost.png", dpi=900)

#plt.title("");

In [ ]:
# Convert dictionary to pandas Series
# Convert dictionary to pandas Series
series = pd.Series(y_test)

# Save the Series to a CSV file as a single line of comma-separated values
series.to_csv('y_test.csv', index=False, header=False)

In [ ]:
# Filter the row where Name is 'XGBoost'
xgboost_row = models_df[models_df['Name'] == 'XGBoost']

# Extract the 'Prob y' value (this is still a single value in a series)
prob_y_series = xgboost_row['Prob y']

# Save the Series to a CSV file
prob_y_series.to_csv('xgboost_prob_y.csv', index=False, header=False)